### challenge.mp4 비디오를 로딩해서 영상을 프레임별 저장하기
1. 비디오 로딩하기
2. 프레임별 이미지 저장하기

In [ ]:
# code check ok
import cv2
import numpy as np

# 동영상 끝나면 종료
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        ret,frame = capture.read()
        if ret == False:
            print('동영상 끝')
            break
        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        frame = cv2.putText(frame,f'{frame_n_count:03d} / {frame_n_total:04d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        cv2.imshow('Lane', frame)

    capture.release()
    cv2.destroyAllWindows()

동영상 끝


In [ ]:
# code check ok
import cv2
import numpy as np

# 동영상 끝나면 처음으로
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        # 프레임 끝이 나오면 프레임No를 처음으로
        if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
            capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

        ret,frame = capture.read()
        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        frame = cv2.putText(frame,f'{frame_n_count:03d} / {frame_n_total:04d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        cv2.imshow('Lane', frame)

    capture.release()
    cv2.destroyAllWindows()

### 동영상파일 그림으로 저장하기

In [ ]:
# code check ok
import cv2
import numpy as np

if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    # int(capture.get(cv2.CAP_PROP_POS_FRAMES)) : 현재 프레임
    # int(capture.get(cv2.CAP_PROP_FRAME_COUNT)) : 총 프레임 수
    while cv2.waitKey(33) < 0:
        ret,frame = capture.read()
        if ret == False:
            print('동영상 끝')
            break
        cv2.imshow('Lane', frame)
        frame_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_count % 10 == 0: # 10프레임마다 한장씩 저장
           #cv2.imwrite('파일이름', 이미지객체)
            cv2.imwrite(f'videos/frame_{frame_count:03d}.jpg', frame)
    capture.release()
    cv2.destroyAllWindows()

동영상 끝


### 흰색, 노란색 차선 추출

In [ ]:
# code check ok
import cv2
import numpy as np

white_min = (150, 150, 150)
white_max = (255, 255, 255)
h_min = 10; h_max = 40
s_min = 100; s_max = 255
v_min = 100; v_max = 255
yellow_min = (h_min, s_min, v_min)
yellow_max = (h_max, s_max, v_max)


# 흰색과 노란색을 추출하여 리턴
def filter_colors(image):
    #1. 흰색 차선 추출 : RGB에서 원하는 색 추출
    white_mask = cv2.inRange(image, white_min, white_max)
    white_image = cv2.bitwise_and(image,image,mask=white_mask)

    #2. 노란색 차선 추출 : HSV에서 노란색 추출
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    yellow_mask = cv2.inRange(image_hsv, yellow_min, yellow_max)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)
    
    #3. 흰색추출결과+노란색추출결과 병합(merge)
    image_result = cv2.addWeighted(white_image, 1.0, yellow_image, 1.0, 0.0)

    return image_result


# 동영상 끝나면 종료
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        # 프레임 끝이 나오면 프레임No를 처음으로
        if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
            capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret,frame = capture.read()
        
        #####################흰색, 노란색 차선 출력 ####################
        frame_result = filter_colors(frame)
        ##################### 프레임수 출력 ############################
        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        frame = cv2.putText(frame_result,f'{frame_n_count:03d} / {frame_n_total:04d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        ###############################################################
        cv2.imshow('Lane', frame_result)

    capture.release()
    cv2.destroyAllWindows()

## fillpoly를 사용하여 원하는 영역 추출하기

In [ ]:
# code check ok
import cv2
import numpy as np
height, width, channel = (720,1280,3)
img_src = cv2.imread('videos/frame_010.jpg',cv2.IMREAD_COLOR)
###################### fillpoly적용하기 ########################

img_mask = np.zeros_like(img_src)
trap_bottom_width = 0.8
trap_top_width = 0.1
trap_height = 0.45

if img_src.ndim > 2 : # color
    mask_color = (255,255,255)
else:  # 1channel : gray
    mask_color = 255


# 꼭지점들 : vertices
                    # x,y 
vertices = np.array([[width*(1-trap_bottom_width)//2, height],
                     [width*(1-trap_top_width)//2, int(height*(1-trap_height))],
                     [width - width*(1-trap_top_width)//2,int(height*(1-trap_height))],
                     [width - width*(1-trap_bottom_width)//2, height]],dtype=np.int32)
vertices = vertices.reshape((-1,1,2))

cv2.fillPoly(img_mask, [vertices], mask_color)
img_src = cv2.bitwise_and(img_src,img_mask)

cv2.imshow('mask',img_mask)
cv2.imshow('src',img_src)
cv2.waitKey()
cv2.destroyAllWindows()


## 도로영상에 원하는 영역만 추출하기

In [ ]:
#code check ok
import cv2
import numpy as np

def region_of_interest(image):
    trap_bottom_width = 0.8
    trap_top_width = 0.1
    trap_height = 0.42

    img_mask = np.zeros_like(image)
    if image.ndim > 2 : # color
        mask_color = (255,255,255)
    else:  # 1channel : gray
        mask_color = 255

    height, width = image.shape[:2]
    # 꼭지점들 : vertices
                        # x,y 
    vertices = np.array([[width*(1-trap_bottom_width)//2, height],
                        [width*(1-trap_top_width)//2, int(height*(1-trap_height))],
                        [width - width*(1-trap_top_width)//2,int(height*(1-trap_height))],
                        [width - width*(1-trap_bottom_width)//2, height]],dtype=np.int32)
    vertices = vertices.reshape((-1,1,2))

    cv2.fillPoly(img_mask, [vertices], mask_color)
    #1. 사다리꼴만 출력
    image = cv2.bitwise_and(image,img_mask)

    # 2. 반투명 이미지 그리기
    #image = cv2.addWeighted(image, 1.0, img_mask, 0.3, 0.0)

    # 3. 사각형 그리기 : polylines
    #image = cv2.polylines(image, [vertices], True, (0,255,255),2)
    return image

# 동영상 끝나면 종료
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        # 프레임 끝이 나오면 프레임No를 처음으로
        if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
            capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret,frame = capture.read()
        
        ##################### 원하는 영역 추출 ####################
        frame_roi  = region_of_interest(frame)
        ##################### 프레임수 출력 ############################
        frame_result = frame_roi
        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        frame = cv2.putText(frame_result,f'{frame_n_count:03d} / {frame_n_total:04d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        ###############################################################
        cv2.imshow('Lane', frame_result)

    capture.release()
    cv2.destroyAllWindows()

## 합치기

In [ ]:
# code check ok
import cv2
import numpy as np

white_min = (150, 150, 150)
white_max = (255, 255, 255)
h_min = 10; h_max = 40
s_min = 100; s_max = 255
v_min = 100; v_max = 255
yellow_min = (h_min, s_min, v_min)
yellow_max = (h_max, s_max, v_max)


# 흰색과 노란색을 추출하여 리턴
def filter_colors(image):
    #1. 흰색 차선 추출 : RGB에서 원하는 색 추출
    white_mask = cv2.inRange(image, white_min, white_max)
    white_image = cv2.bitwise_and(image,image,mask=white_mask)

    #2. 노란색 차선 추출 : HSV에서 노란색 추출
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    yellow_mask = cv2.inRange(image_hsv, yellow_min, yellow_max)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)
    
    #3. 흰색추출결과+노란색추출결과 병합(merge)
    image_result = cv2.addWeighted(white_image, 1.0, yellow_image, 1.0, 0.0)

    return image_result


def region_of_interest(image):
    trap_bottom_width = 0.8
    trap_top_width = 0.1
    trap_height = 0.45

    img_mask = np.zeros_like(image)
    if image.ndim > 2 : # color
        mask_color = (255,255,255)
    else:  # 1channel : gray
        mask_color = 255

    height, width = image.shape[:2]
    # 꼭지점들 : vertices
                        # x,y 
    vertices = np.array([[width*(1-trap_bottom_width)//2, height],
                        [width*(1-trap_top_width)//2, int(height*(1-trap_height))],
                        [width - width*(1-trap_top_width)//2,int(height*(1-trap_height))],
                        [width - width*(1-trap_bottom_width)//2, height]],dtype=np.int32)
    vertices = vertices.reshape((-1,1,2))

    cv2.fillPoly(img_mask, [vertices], mask_color)
    #1. 사다리꼴만 출력
    image = cv2.bitwise_and(image,img_mask)

    # 2. 반투명 이미지 그리기
    #image = cv2.addWeighted(image, 1.0, img_mask, 0.3, 0.0)

    # 3. 사각형 그리기 : polylines
    #image = cv2.polylines(image, [vertices], True, (0,255,255),2)
    return image

# 동영상 끝나면 종료
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        # 프레임 끝이 나오면 프레임No를 처음으로
        if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
            capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret,frame = capture.read()

        #####################흰색, 노란색 차선 출력 ####################
        frame_filter = filter_colors(frame)
        
        ##################### 원하는 영역 추출 ####################
        frame_roi  = region_of_interest(frame_filter)
        ##################### 프레임수 출력 ############################
        frame_result = frame_roi
        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        cv2.putText(frame_result,f'{frame_n_count:03d} / {frame_n_total:04d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        ###############################################################
        cv2.imshow('Lane', frame_result)

    capture.release()
    cv2.destroyAllWindows()

### 에지구하기 
- 그래이변환
- 노이즈제거
- Thresholding
- Canny Edge구하기

In [ ]:
# code check ok
import cv2
import numpy as np

white_min = (150, 150, 150)
white_max = (255, 255, 255)
h_min = 10; h_max = 40
s_min = 100; s_max = 255
v_min = 100; v_max = 255
yellow_min = (h_min, s_min, v_min)
yellow_max = (h_max, s_max, v_max)


# 흰색과 노란색을 추출하여 리턴
def filter_colors(image):
    #1. 흰색 차선 추출 : RGB에서 원하는 색 추출
    white_mask = cv2.inRange(image, white_min, white_max)
    white_image = cv2.bitwise_and(image,image,mask=white_mask)

    #2. 노란색 차선 추출 : HSV에서 노란색 추출
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    yellow_mask = cv2.inRange(image_hsv, yellow_min, yellow_max)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)
    
    #3. 흰색추출결과+노란색추출결과 병합(merge)
    image_result = cv2.addWeighted(white_image, 1.0, yellow_image, 1.0, 0.0)

    return image_result


def region_of_interest(image):
    trap_bottom_width = 0.8
    trap_top_width = 0.1
    trap_height = 0.45

    img_mask = np.zeros_like(image)
    if image.ndim > 2 : # color
        mask_color = (255,255,255)
    else:  # 1channel : gray
        mask_color = 255

    height, width = image.shape[:2]
    # 꼭지점들 : vertices
                        # x,y 
    vertices = np.array([[width*(1-trap_bottom_width)//2, height],
                        [width*(1-trap_top_width)//2, int(height*(1-trap_height))],
                        [width - width*(1-trap_top_width)//2,int(height*(1-trap_height))],
                        [width - width*(1-trap_bottom_width)//2, height]],dtype=np.int32)
    vertices = vertices.reshape((-1,1,2))

    cv2.fillPoly(img_mask, [vertices], mask_color)
    #1. 사다리꼴만 출력
    image = cv2.bitwise_and(image,img_mask)

    # 2. 반투명 이미지 그리기
    #image = cv2.addWeighted(image, 1.0, img_mask, 0.3, 0.0)

    # 3. 사각형 그리기 : polylines
    #image = cv2.polylines(image, [vertices], True, (0,255,255),2)
    return image

# 동영상 끝나면 종료
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        # 프레임 끝이 나오면 프레임No를 처음으로
        if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
            capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret,frame = capture.read()

        #####################흰색, 노란색 차선 출력 ####################
        frame_filter = filter_colors(frame)
        # print(frame_filter.shape) #--> (720, 1280, 3) :칼라 영상임
        # cv2.imshow('filter',frame_filter)
        ################ 노이즈제거, Thresholding 적용 #################
        # gray로 변환
        frame_gray = cv2.cvtColor(frame_filter, cv2.COLOR_BGR2GRAY)
        #cv2.imshow('gray',frame_gray)
        # 노이즈 제거 : Blur --> GaussianBlur적용
        frame_gray = cv2.GaussianBlur(frame_gray, (5,5), 0) # 마스크크기가 커지면 더 흐려짐
        #cv2.imshow('gray+blur',frame_gray)
        ret, frame_binary = cv2.threshold(frame_gray, 0,255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
        #cv2.imshow('threshold',frame_binary)
        # 에지 구하기
        frame_canny = cv2.Canny(frame_binary, 50,150)
        # cv2.imshow('canny edge',frame_canny)
        ##################### 원하는 영역 추출 #########################
        frame_roi  = region_of_interest(frame_canny)
        #cv2.imshow('roi',frame_roi)
        ##################### 프레임수 출력 ############################
        if frame_roi.ndim < 3 :
            frame_result = cv2.cvtColor(frame_roi, cv2.COLOR_GRAY2BGR)
        else:
            frame_result = frame_roi.copy()

        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        cv2.putText(frame_result, f'{frame_n_count:03d} / {frame_n_total:04d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        ###############################################################
        cv2.imshow('Lane', frame_result)

    capture.release()
    cv2.destroyAllWindows()

### CannyEdge 결과영상 저장

In [ ]:
#code check ok
import cv2
import numpy as np

white_min = (150, 150, 150)
white_max = (255, 255, 255)
h_min = 10; h_max = 40
s_min = 100; s_max = 255
v_min = 100; v_max = 255
yellow_min = (h_min, s_min, v_min)
yellow_max = (h_max, s_max, v_max)


# 흰색과 노란색을 추출하여 리턴
def filter_colors(image):
    #1. 흰색 차선 추출 : RGB에서 원하는 색 추출
    white_mask = cv2.inRange(image, white_min, white_max)
    white_image = cv2.bitwise_and(image,image,mask=white_mask)

    #2. 노란색 차선 추출 : HSV에서 노란색 추출
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    yellow_mask = cv2.inRange(image_hsv, yellow_min, yellow_max)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)
    
    #3. 흰색추출결과+노란색추출결과 병합(merge)
    image_result = cv2.addWeighted(white_image, 1.0, yellow_image, 1.0, 0.0)

    return image_result

def region_of_interest(image):
    trap_bottom_width = 0.8
    trap_top_width = 0.1
    trap_height = 0.45

    img_mask = np.zeros_like(image)
    if image.ndim > 2 : # color
        mask_color = (255,255,255)
    else:  # 1channel : gray
        mask_color = 255

    height, width = image.shape[:2]
    # 꼭지점들 : vertices
                        # x,y 
    vertices = np.array([[width*(1-trap_bottom_width)//2, height],
                        [width*(1-trap_top_width)//2, int(height*(1-trap_height))],
                        [width - width*(1-trap_top_width)//2,int(height*(1-trap_height))],
                        [width - width*(1-trap_bottom_width)//2, height]],dtype=np.int32)
    vertices = vertices.reshape((-1,1,2))

    cv2.fillPoly(img_mask, [vertices], mask_color)
    #1. 사다리꼴만 출력
    image = cv2.bitwise_and(image,img_mask)

    # 2. 반투명 이미지 그리기
    #image = cv2.addWeighted(image, 1.0, img_mask, 0.3, 0.0)

    # 3. 사각형 그리기 : polylines
    #image = cv2.polylines(image, [vertices], True, (0,255,255),2)
    return image

# 동영상 끝나면 종료
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        # 프레임 끝이 나오면 프레임No를 처음으로
        if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
            capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret,frame = capture.read()

        #####################흰색, 노란색 차선 출력 ####################
        frame_filter = filter_colors(frame)
        # print(frame_filter.shape) #--> (720, 1280, 3) :칼라 영상임
        # cv2.imshow('filter',frame_filter)
        ################ 노이즈제거, Thresholding 적용 #################
        # gray로 변환
        frame_gray = cv2.cvtColor(frame_filter, cv2.COLOR_BGR2GRAY)
        #cv2.imshow('gray',frame_gray)
        # 노이즈 제거 : Blur --> GaussianBlur적용
        frame_gray = cv2.GaussianBlur(frame_gray, (5,5), 0) # 마스크크기가 커지면 더 흐려짐
        #cv2.imshow('gray+blur',frame_gray)
        ret, frame_binary = cv2.threshold(frame_gray, 0,255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
        #cv2.imshow('threshold',frame_binary)
        # 에지 구하기
        frame_canny = cv2.Canny(frame_binary, 50,150)
        # cv2.imshow('canny edge',frame_canny)
        ##################### 원하는 영역 추출 #########################
        frame_roi  = region_of_interest(frame_canny)
        #cv2.imshow('roi',frame_roi)
        ##################### 프레임수 출력 ############################
        if frame_roi.ndim < 3 :
            frame_result = cv2.cvtColor(frame_roi, cv2.COLOR_GRAY2BGR)
        else:
            frame_result = frame_roi.copy()
        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        cv2.putText(frame_result, f'{frame_n_count:03d} / {frame_n_total:04d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        ###############################################################
        cv2.imshow('Lane', frame_result)
        ####################### 저장하기 ############################
        if frame_n_count % 10 == 0: # 10프레임마다 한장씩 저장
            cv2.imwrite(f'videos/frame_canny_{frame_n_count:03d}.jpg', frame_roi)

    capture.release()
    cv2.destroyAllWindows()

### HoughLine 그리기

In [ ]:
#code check ok
import cv2
import numpy as np

white_min = (150, 150, 150)
white_max = (255, 255, 255)
h_min = 10; h_max = 40
s_min = 100; s_max = 255
v_min = 100; v_max = 255
yellow_min = (h_min, s_min, v_min)
yellow_max = (h_max, s_max, v_max)

# for HoughLines
rho = 2
theta = 1 * np.pi / 180 # 1도
threshold = 15
min_line_len = 10
max_line_gap = 20


# 흰색과 노란색을 추출하여 리턴
def filter_colors(image):
    #1. 흰색 차선 추출 : RGB에서 원하는 색 추출
    white_mask = cv2.inRange(image, white_min, white_max)
    white_image = cv2.bitwise_and(image,image,mask=white_mask)

    #2. 노란색 차선 추출 : HSV에서 노란색 추출
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    yellow_mask = cv2.inRange(image_hsv, yellow_min, yellow_max)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)
    
    #3. 흰색추출결과+노란색추출결과 병합(merge)
    image_result = cv2.addWeighted(white_image, 1.0, yellow_image, 1.0, 0.0)

    return image_result


def region_of_interest(image):
    trap_bottom_width = 0.8
    trap_top_width = 0.1
    trap_height = 0.45

    img_mask = np.zeros_like(image)
    if image.ndim > 2 : # color
        mask_color = (255,255,255)
    else:  # 1channel : gray
        mask_color = 255

    height, width = image.shape[:2]
    # 꼭지점들 : vertices
                        # x,y 
    vertices = np.array([[width*(1-trap_bottom_width)//2, height],
                        [width*(1-trap_top_width)//2, int(height*(1-trap_height))],
                        [width - width*(1-trap_top_width)//2,int(height*(1-trap_height))],
                        [width - width*(1-trap_bottom_width)//2, height]],dtype=np.int32)
    vertices = vertices.reshape((-1,1,2))

    cv2.fillPoly(img_mask, [vertices], mask_color)
    #1. 사다리꼴만 출력
    image = cv2.bitwise_and(image,img_mask)

    # 2. 반투명 이미지 그리기
    #image = cv2.addWeighted(image, 1.0, img_mask, 0.3, 0.0)

    # 3. 사각형 그리기 : polylines
    #image = cv2.polylines(image, [vertices], True, (0,255,255),2)
    return image

# 동영상 끝나면 종료
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        # 프레임 끝이 나오면 프레임No를 처음으로
        if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
            capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret,frame = capture.read()

        #####################흰색, 노란색 차선 출력 ####################
        frame_filter = filter_colors(frame)
        # print(frame_filter.shape) #--> (720, 1280, 3) :칼라 영상임
        # cv2.imshow('filter',frame_filter)
        ################ 노이즈제거, Thresholding 적용 #################
        # gray로 변환
        frame_gray = cv2.cvtColor(frame_filter, cv2.COLOR_BGR2GRAY)
        #cv2.imshow('gray',frame_gray)
        # 노이즈 제거 : Blur --> GaussianBlur적용
        frame_gray = cv2.GaussianBlur(frame_gray, (5,5), 0) # 마스크크기가 커지면 더 흐려짐
        #cv2.imshow('gray+blur',frame_gray)
        ret, frame_binary = cv2.threshold(frame_gray, 0,255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
        #cv2.imshow('threshold',frame_binary)
        # 에지 구하기
        frame_canny = cv2.Canny(frame_binary, 50,150)
        # cv2.imshow('canny edge',frame_canny)
        ##################### 원하는 영역 추출 #########################
        frame_roi  = region_of_interest(frame_canny)
        #cv2.imshow('roi',frame_roi)
        ###################### HoughLine 그리기 #######################
        lines = cv2.HoughLinesP(frame_roi, rho, theta, threshold, np.array([]), 
                        minLineLength = min_line_len, maxLineGap = max_line_gap)
        for i, line in enumerate(lines):
            cv2.line(frame, (line[0][0], line[0][1]),(line[0][2], line[0][3]), (0,255,0),2)

        ##################### 프레임수 출력 ############################
        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        cv2.putText(frame, f'{frame_n_count:03d} / {frame_n_total:04d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        ###############################################################
        cv2.imshow('Lane', frame)

    capture.release()
    cv2.destroyAllWindows()

## Canny 영상으로 HoughLine추출후 대표 차선 그리기

In [ ]:
# code check ok
import cv2
import numpy as np

# for houghLines
rho = 2
theta = 1 * np.pi / 180
threshold = 15
min_line_len = 10
max_line_gap = 20

# 
trap_bottom_width = 0.8
trap_top_width = 0.1
trap_height = 0.45

def draw_lines(image, lines, color=[255,0,0], thickness=12):
    ### 예외처리 #####
    if lines is None:
        return
    if len(lines) == 0:
        return  
    
    draw_right = True
    draw_left = True

    ### 허프라인들의 모임가운데 대표 라인 찾기
    # 차선은 반드시 어느정도의 기울기를 가져야 함
    # 기울기가 slope_threshold보다 커야함(좌우를 적용) : abs(slope) > slope_threshold
    slope_threshold = 0.5
    slopes=[]
    new_lines=[]
    # 차선과 관련없는 라인들 제거
    for line in lines:
        x1,y1,x2,y2 = line[0] # line = [[x1,y1,x2,y2]]

        #slope계산
        if x2-x1 == 0.: #분모가 0, 예외처리필요
            slope = 999.
        else:
            slope = (y2-y1) / (x2-x1)

        if abs(slope) > slope_threshold:
            slopes.append(slope)
            new_lines.append(line)
    
    lines = new_lines

    # 라인들중에 (왼쪽라인구성하는 선) vs (오른쪽 라인을 구성하는 선) 구분(나누기)
    # 이미지의 가운데(img_x_center = width//2)를 기준으로 왼쪽,오른쪽 구분

    right_lines=[]
    left_lines=[]

    # (height, width) = image.shape[:2]
    image_x_center = image.shape[1]/2
    for i, line in enumerate(lines):
        x1,y1,x2,y2 = line[0]   # 점2개의 x좌표가 왼쪽/오른쪽에 있어야함
        if slopes[i] > 0 and x1 > image_x_center and x2 > image_x_center:
            right_lines.append(line)
        if slopes[i] < 0 and x1 < image_x_center and x2 < image_x_center:
            left_lines.append(line)


    # 오른쪽/왼쪽 라인들을 가지고 오차가 가장 적은 선을 하나 추출하기
    # liner regression을 적용하여 오차가 가장 적은 선 추출
    # 오른쪽 라인
    right_lines_x=[]
    right_lines_y=[]
    for line in right_lines:
        x1,y1,x2,y2 = line[0]
        right_lines_x.append(x1)
        right_lines_x.append(x2)
        
        right_lines_y.append(y1)
        right_lines_y.append(y2)

    if len(right_lines_x) > 0:
        right_m, right_b = np.polyfit(right_lines_x, right_lines_y,1) # y = m*x +b
    else:
        right_m, right_b = 1, 1
        draw_right = False

    # 왼쪽 라인
    left_lines_x=[]
    left_lines_y=[]
    for line in left_lines:
        x1,y1,x2,y2 = line[0]
        left_lines_x.append(x1)
        left_lines_x.append(x2)
        
        left_lines_y.append(y1)
        left_lines_y.append(y2)

    if len(left_lines_x) > 0:
        left_m, left_b = np.polyfit(left_lines_x, left_lines_y,1) # y = m*x +b
    else:
        left_m, left_b = 1, 1
        draw_left = False

    # 오른쪽, 왼쪽 각 2개의 x의 좌표를 구해야 함
    # 우리는 기본적으로 y는 고정해놓은 상태임
    y1 = int(image.shape[0])
    y2 = int(image.shape[0] * (1-trap_height))

    right_x1 = int((y1-right_b) / right_m)
    right_x2 = int((y2-right_b) / right_m)

    left_x1 = int((y1-left_b) / left_m)
    left_x2 = int((y2-left_b) / left_m)

    # 오른쪽, 왼쪽 대표 라인 그리기
    if draw_right:
        cv2.line(image, (right_x1, y1), (right_x2,y2), color=color, thickness=thickness)
    if draw_left:
        cv2.line(image, (left_x1, y1), (left_x2,y2), color=color, thickness=thickness)
    
    return image


if __name__ == '__main__':
    # 사다리꼴 안에 있는 canny edge 결과물
    frame_roi = cv2.imread('videos/frame_canny_030.jpg', cv2.IMREAD_GRAYSCALE)
    height, width = frame_roi.shape[:2]

    lines = cv2.HoughLinesP(frame_roi, rho, theta, threshold, np.array([]), \
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    

    # print(len(lines))
    # print(lines)

    frame_dst = cv2.cvtColor(frame_roi, cv2.COLOR_GRAY2BGR)
    # for i, line in enumerate(lines):
    #         cv2.line(frame_dst, (line[0][0], line[0][1]),(line[0][2], line[0][3]), (0,255,0),2)

    image_zero = np.zeros((height,width,3), dtype=np.uint8)
    ##### HoughLines에서 추출된 선을 가지고 좌우 차선을 대표하는 대표선 추출 함수 생성 #####
    image_zero = draw_lines(image_zero, lines)
    # 원본영상에 추출된 차선 겹치기
    frame_dst  = cv2.addWeighted(frame_dst, 0.8, image_zero, 1.0, 0.0)
    ###################################################################################
    
    cv2.imshow('frame_roi',frame_dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [ ]:
# code check ok
import cv2
import numpy as np

white_min = (150, 150, 150)
white_max = (255, 255, 255)
h_min = 10; h_max = 40
s_min = 100; s_max = 255
v_min = 100; v_max = 255
yellow_min = (h_min, s_min, v_min)
yellow_max = (h_max, s_max, v_max)

# for HoughLines
rho = 2
theta = 1 * np.pi / 180 # 1도
threshold = 15
min_line_len = 10
max_line_gap = 20

# 사다리꼴 그리기
trap_bottom_width = 0.8
trap_top_width = 0.1
trap_height = 0.42

def draw_lines(image, lines, color=[255,0,0], thickness=12):
    ### 예외처리 #####
    if lines is None:
        return
    if len(lines) == 0:
        return  
    
    draw_right = True
    draw_left = True

    ### 허프라인들의 모임가운데 대표 라인 찾기
    # 차선은 반드시 어느정도의 기울기를 가져야 함
    # 기울기가 slope_threshold보다 커야함(좌우를 적용) : abs(slope) > slope_threshold
    slope_threshold = 0.5
    slopes=[]
    new_lines=[]
    # 차선과 관련없는 라인들 제거
    for line in lines:
        x1,y1,x2,y2 = line[0] # line = [[x1,y1,x2,y2]]

        #slope계산
        if x2-x1 == 0.: #분모가 0, 예외처리필요
            slope = 999.
        else:
            slope = (y2-y1) / (x2-x1)

        if abs(slope) > slope_threshold:
            slopes.append(slope)
            new_lines.append(line)
    
    lines = new_lines

    # 라인들중에 (왼쪽라인구성하는 선) vs (오른쪽 라인을 구성하는 선) 구분(나누기)
    # 이미지의 가운데(img_x_center = width//2)를 기준으로 왼쪽,오른쪽 구분

    right_lines=[]
    left_lines=[]

    # (height, width) = image.shape[:2]
    image_x_center = image.shape[1]/2
    for i, line in enumerate(lines):
        x1,y1,x2,y2 = line[0]   # 점2개의 x좌표가 왼쪽/오른쪽에 있어야함
        if slopes[i] > 0 and x1 > image_x_center and x2 > image_x_center:
            right_lines.append(line)
        if slopes[i] < 0 and x1 < image_x_center and x2 < image_x_center:
            left_lines.append(line)


    # 오른쪽/왼쪽 라인들을 가지고 오차가 가장 적은 선을 하나 추출하기
    # liner regression을 적용하여 오차가 가장 적은 선 추출
    # 오른쪽 라인
    right_lines_x=[]
    right_lines_y=[]
    for line in right_lines:
        x1,y1,x2,y2 = line[0]
        right_lines_x.append(x1)
        right_lines_x.append(x2)
        
        right_lines_y.append(y1)
        right_lines_y.append(y2)

    if len(right_lines_x) > 0:
        right_m, right_b = np.polyfit(right_lines_x, right_lines_y,1) # y = m*x +b
    else:
        right_m, right_b = 1, 1
        draw_right = False

    # 왼쪽 라인
    left_lines_x=[]
    left_lines_y=[]
    for line in left_lines:
        x1,y1,x2,y2 = line[0]
        left_lines_x.append(x1)
        left_lines_x.append(x2)
        
        left_lines_y.append(y1)
        left_lines_y.append(y2)

    if len(left_lines_x) > 0:
        left_m, left_b = np.polyfit(left_lines_x, left_lines_y,1) # y = m*x +b
    else:
        left_m, left_b = 1, 1
        draw_left = False

    # 오른쪽, 왼쪽 각 2개의 x의 좌표를 구해야 함
    # 우리는 기본적으로 y는 고정해놓은 상태임
    y1 = int(image.shape[0])
    y2 = int(image.shape[0] * (1-trap_height))

    right_x1 = int((y1-right_b) / right_m)
    right_x2 = int((y2-right_b) / right_m)

    left_x1 = int((y1-left_b) / left_m)
    left_x2 = int((y2-left_b) / left_m)

    # 오른쪽, 왼쪽 대표 라인 그리기
    if draw_right:
        cv2.line(image, (right_x1, y1), (right_x2,y2), color=color, thickness=thickness)
    if draw_left:
        cv2.line(image, (left_x1, y1), (left_x2,y2), color=color, thickness=thickness)
    
    return image


# 흰색과 노란색을 추출하여 리턴
def filter_colors(image):
    #1. 흰색 차선 추출 : RGB에서 원하는 색 추출
    white_mask = cv2.inRange(image, white_min, white_max)
    white_image = cv2.bitwise_and(image,image,mask=white_mask)

    #2. 노란색 차선 추출 : HSV에서 노란색 추출
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    yellow_mask = cv2.inRange(image_hsv, yellow_min, yellow_max)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)
    
    #3. 흰색추출결과+노란색추출결과 병합(merge)
    image_result = cv2.addWeighted(white_image, 1.0, yellow_image, 1.0, 0.0)

    return image_result


def region_of_interest(image):
    img_mask = np.zeros_like(image)
    if image.ndim > 2 : # color
        mask_color = (255,255,255)
    else:  # 1channel : gray
        mask_color = 255

    height, width = image.shape[:2]
    # 꼭지점들 : vertices
                        # x,y 
    vertices = np.array([[width*(1-trap_bottom_width)//2, height],
                        [width*(1-trap_top_width)//2, int(height*(1-trap_height))],
                        [width - width*(1-trap_top_width)//2,int(height*(1-trap_height))],
                        [width - width*(1-trap_bottom_width)//2, height]],dtype=np.int32)
    vertices = vertices.reshape((-1,1,2))

    cv2.fillPoly(img_mask, [vertices], mask_color)
    #1. 사다리꼴만 출력
    image = cv2.bitwise_and(image,img_mask)

    # 2. 반투명 이미지 그리기
    #image = cv2.addWeighted(image, 1.0, img_mask, 0.3, 0.0)

    # 3. 사각형 그리기 : polylines
    #image = cv2.polylines(image, [vertices], True, (0,255,255),2)
    return image

# 동영상 끝나면 종료
if __name__ == '__main__':
    capture = cv2.VideoCapture('videos/challenge.mp4')
    frame_n_total = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    while cv2.waitKey(33) < 0:
        # 프레임 끝이 나오면 프레임No를 처음으로
        if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
            capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret,frame = capture.read()

        #####################흰색, 노란색 차선 출력 ####################
        frame_filter = filter_colors(frame)
        # print(frame_filter.shape) #--> (720, 1280, 3) :칼라 영상임
        # cv2.imshow('filter',frame_filter)
        ################ 노이즈제거, Thresholding 적용 #################
        # gray로 변환
        frame_gray = cv2.cvtColor(frame_filter, cv2.COLOR_BGR2GRAY)
        #cv2.imshow('gray',frame_gray)
        # 노이즈 제거 : Blur --> GaussianBlur적용
        frame_gray = cv2.GaussianBlur(frame_gray, (5,5), 0) # 마스크크기가 커지면 더 흐려짐
        #cv2.imshow('gray+blur',frame_gray)
        ret, frame_binary = cv2.threshold(frame_gray, 0,255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
        #cv2.imshow('threshold',frame_binary)
        # 에지 구하기
        frame_canny = cv2.Canny(frame_binary, 50,150)
        # cv2.imshow('canny edge',frame_canny)
        ##################### 원하는 영역 추출 #########################
        frame_roi  = region_of_interest(frame_canny)
        #cv2.imshow('roi',frame_roi)
        ###################### HoughLine 그리기 #######################
        lines = cv2.HoughLinesP(frame_roi, rho, theta, threshold, np.array([]), 
                        minLineLength = min_line_len, maxLineGap = max_line_gap)
        
        image_lines = np.zeros_like(frame)
        ##### HoughLines에서 추출된 선을 가지고 좌우 차선을 대표하는 대표선 추출 함수 생성 #####
        image_lines = draw_lines(image_lines, lines)
        frame = cv2.addWeighted(frame, 0.8, image_lines, 1.0, 0.0)
        ##################### 프레임수 출력 ############################
        frame_n_count = int(capture.get(cv2.CAP_PROP_POS_FRAMES))
        cv2.putText(frame, f'{frame_n_count:03d} / {frame_n_total:03d}', (10,25), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)
        ###############################################################
        cv2.imshow('Lane', frame)

    capture.release()
    cv2.destroyAllWindows()